In [1]:
import pandas as pd
import datetime as dt
#  import numpy as np
import json
import requests


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
pol_13_16_df = pd.read_csv('landing_data/pol_13_16_df.csv')
pol_13_16_df.head()

,State,State_Abbr,id,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
0,Alabama,AL,0.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.259091,0.80,2013.0
1,Alabama,AL,1.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.266667,0.50,2013.0
2,Alabama,AL,4.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.305625,0.47,2013.0
3,Alabama,AL,5.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.400000,0.50,2013.0
4,Alabama,AL,8.0,2013-12-03,Parts per billion,14.912500,Parts per million,0.008167,Parts per billion,0.305263,11.0,3.0,Parts per million,0.216818,0.35,2013.0


In [3]:
# drop columns not needed - I hope.
pol_13_16_df.drop(columns=['id', 'Date_Local', 'NO2_Units', \
                           'O3_Units', 'SO2_Units', 'CO_Units'], inplace=True)
pol_13_16_df.head()

,State,State_Abbr,NO2_Mean,O3_Mean,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Mean,CO 1st Max Value,year
0,Alabama,AL,17.208333,0.013542,0.313636,11.0,1.0,0.259091,0.80,2013.0
1,Alabama,AL,17.208333,0.013542,0.313636,11.0,1.0,0.266667,0.50,2013.0
2,Alabama,AL,20.687500,0.009375,0.530000,11.0,3.0,0.305625,0.47,2013.0
3,Alabama,AL,20.687500,0.009375,0.530000,11.0,3.0,0.400000,0.50,2013.0
4,Alabama,AL,14.912500,0.008167,0.305263,11.0,3.0,0.216818,0.35,2013.0


Go thru and generate stats off of pol_13_16_df dataframe

In [8]:
# df.groupby(['col5','col2']).reset_index()

pol_groupby_min_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).min().groupby('State_Abbr', as_index=False)
pol_groupby_min_df.head()

,year,State_Abbr,State,NO2_Mean,O3_Mean,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Mean,CO 1st Max Value
0,2013.0,AL,Alabama,3.770833,0.000000,-0.150000,0.0,0.0,0.058500,0.100
1,2013.0,AR,Arkansas,1.675000,0.003292,0.204167,0.0,0.0,0.033333,0.100
2,2013.0,AZ,Arizona,2.895833,0.002000,-0.033333,0.0,0.0,0.039130,0.100
3,2013.0,CA,California,-0.073684,0.000000,-0.979167,0.0,0.0,-0.200000,-0.200
4,2013.0,CO,Colorado,-0.095238,0.000083,-0.041667,0.0,0.0,-0.195833,0.000
5,2013.0,CT,Connecticut,0.000000,0.000958,-0.065217,0.0,0.0,0.090435,0.100
6,2013.0,DC,District Of Columbia,1.266667,0.000125,-0.952632,0.0,0.0,-0.100000,-0.100
7,2013.0,DE,Delaware,3.295833,0.001667,0.009091,0.0,0.0,0.100000,0.100
8,2013.0,FL,Florida,0.600000,0.008583,-1.000000,0.0,0.0,-0.062500,0.000
9,2013.0,GA,Georgia,1.495833,0.002875,0.100000,0.0,0.0,0.098083,0.180


In [5]:
# - get max

pol_groupby_max_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).max().groupby('State_Abbr', as_index=False)
pol_groupby_max_df.head()


,year,State_Abbr,State,NO2_Mean,O3_Mean,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Mean,CO 1st Max Value
0,2013.0,AL,Alabama,25.329167,0.026333,4.618182,23.0,50.0,0.445833,1.210
1,2013.0,AR,Arkansas,26.845833,0.049000,3.129167,23.0,11.0,1.083333,1.552
2,2013.0,AZ,Arizona,47.375000,0.059750,3.791667,23.0,13.0,1.333333,3.239
3,2013.0,CA,California,53.663636,0.071167,13.761905,23.0,104.0,3.158824,6.547
4,2013.0,CO,Colorado,49.173913,0.057100,11.625000,23.0,76.0,1.904167,5.800
5,2013.0,CT,Connecticut,46.250000,0.067250,8.020833,23.0,37.0,1.054167,3.030
6,2013.0,DC,District Of Columbia,35.875000,0.055917,4.920000,23.0,23.0,1.460000,2.300
7,2013.0,DE,Delaware,26.520833,0.047833,3.500000,23.0,11.0,0.520833,1.381
8,2013.0,FL,Florida,20.666667,0.049042,0.625000,22.0,7.0,0.858333,1.200
9,2013.0,GA,Georgia,27.079167,0.047250,1.679167,23.0,20.0,0.812500,1.407


In [ ]:
# - mean

pol_groupby_mean_df = pol_13_16_df.groupby(['year', 'State_Abbr'], as_index=False).mean().reset_index().groupby('State_Abbr', as_index=False)
pol_groupby_mean_df.head(11)

Merge the dataframes

In [ ]:
# - 


In [6]:
stats_df = pd.merge(pol_groupby_min_df, pol_groupby_max_df, how='left', left_on=['year', 'State_Abbr'], right_on=['year', 'State_Abbr'])
stats_df.head(2)

TypeError: Can only merge Series or DataFrame objects, a <class 'pandas.core.groupby.generic.DataFrameGroupBy'> was passed